In [2]:
import cv2
import torch
import numpy as np
import random 
random.seed(1337)

import warnings 
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

from torchvision import transforms 
import torchvision.transforms.functional as TF

import torch.nn as nn

from transformers import Blip2Processor, Blip2ForConditionalGeneration

In [3]:
image_folder = "image_folder"  # Replace with your actual path
caption_file = "captions.txt"        # Your captions file
batch_size = 4
vocab_size = None  # Will infer from captions
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
video_path = "Benchmark-AllVideos-HQ-Encoded-challenge/_a6lFCUYTA4.mp4"

In [5]:
# def extract_frames(video_path):
#     cap = cv2.VideoCapture(video_path)
#     frames = []
    
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         frames.append(frame)
    
#     cap.release()
#     print(f"Total number of frames in the video: {len(frames)}")
    
#     return frames

# frames = extract_frames(video_path)

In [6]:
# def cos_sim_scores(frames):
#     similarities = [] 

#     for i in range(1, len(frames)):
#         prev_frame = frames[i - 1].flatten().astype(np.float32)
#         curr_frame = frames[i].flatten().astype(np.float32)
#         similarity = cosine_similarity([prev_frame], [curr_frame])[0][0]

#         similarities.append(similarity) 

#     return similarities

# similarities = cos_sim_scores(frames)

# plt.figure(figsize=(12, 4))
# plt.plot(similarities, marker='o', label='Cosine Similarity')
# plt.title('Cosine Similarity Between Consecutive Frames')
# plt.xlabel('Frame Index')
# plt.ylabel('Similarity Score')
# plt.grid(True)
# plt.legend()
# plt.tight_layout()
# plt.show()

In [7]:
# def segment_scenes(frames, threshold):
#     segments = []
#     current_segment = [frames[0]]

#     for i in range(1, len(frames)):
#         prev_frame = frames[i - 1].flatten().astype(np.float32)
#         curr_frame = frames[i].flatten().astype(np.float32)
#         similarity = cosine_similarity([prev_frame], [curr_frame])[0][0]

#         if similarity < threshold:
#             segments.append(current_segment)
#             current_segment = [frames[i]]
#         else:
#             current_segment.append(frames[i])

#     segments.append(current_segment)
#     return segments

# threshold=0.965
# segments = segment_scenes(frames, threshold)

In [8]:
# num_segments = len(segments)
# plt.figure(figsize=(10, 4 * num_segments))

# for i, segment in enumerate(segments):
#     first_frame = segment[0]
#     last_frame = segment[-1]

#     # Plot first frame
#     plt.subplot(num_segments, 2, i * 2 + 1)
#     plt.imshow(cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB))
#     plt.title(f'Segment {i+1} - First Frame')
#     plt.axis('off')

#     # Plot last frame
#     plt.subplot(num_segments, 2, i * 2 + 2)
#     plt.imshow(cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB))
#     plt.title(f'Segment {i+1} - Last Frame')
#     plt.axis('off')

# plt.tight_layout()
# plt.show()

In [9]:
# resize so longer side = 224, then pad shorter side 
def resize_with_pad(image, size=224):
    # Resize so the longer side == size
    w, h = image.size
    scale = size / max(w, h)
    new_w, new_h = int(w * scale), int(h * scale)
    
    resized = TF.resize(image, (new_h, new_w), interpolation=transforms.InterpolationMode.BICUBIC)
    
    # Pad to (size, size)
    pad_w = size - new_w
    pad_h = size - new_h
    padding = (pad_w // 2, pad_h // 2, pad_w - pad_w // 2, pad_h - pad_h // 2)  # (left, top, right, bottom)
    padded = TF.pad(resized, padding, fill=0)  # Fill with black
    return padded

In [10]:
# class PatchEmbedding(nn.Module):
#     def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
#         super().__init__()
#         self.num_patches = (img_size // patch_size) ** 2
#         self.patch_embed = nn.Conv2d(
#             in_channels, embed_dim, 
#             kernel_size=patch_size, 
#             stride=patch_size
#         )
        
#     def forward(self, x):
#         x = self.patch_embed(x)  # (batch_size, embed_dim, grid, grid)
#         x = x.flatten(2)         # (batch_size, embed_dim, num_patches)
#         x = x.transpose(1, 2)    # (batch_size, num_patches, embed_dim)
#         return x

# class TransformerEncoderBlock(nn.Module):
#     def __init__(self, embed_dim=768, num_heads=8, mlp_ratio=4.0, dropout=0.1):
#         super().__init__()
#         self.norm1 = nn.LayerNorm(embed_dim)
#         self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
#         self.norm2 = nn.LayerNorm(embed_dim)
#         self.mlp = nn.Sequential(
#             nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),
#             nn.GELU(),
#             nn.Linear(int(embed_dim * mlp_ratio), embed_dim)
#         )
        
#     def forward(self, x):
#         # Self-attention
#         x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
#         # Feed Forward
#         x = x + self.mlp(self.norm2(x))
#         return x

# class TinyVisionTransformer(nn.Module):
#     def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768, depth=6, num_heads=8):
#         super().__init__()
#         self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
#         self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
#         self.pos_embed = nn.Parameter(torch.zeros(1, (img_size // patch_size) ** 2 + 1, embed_dim))
#         self.blocks = nn.ModuleList([
#             TransformerEncoderBlock(embed_dim, num_heads) for _ in range(depth)
#         ])
#         self.norm = nn.LayerNorm(embed_dim)
        
#     def forward(self, x):
#         batch_size = x.size(0)
#         x = self.patch_embed(x)  # (batch_size, num_patches, embed_dim)

#         cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # (batch_size, 1, embed_dim)
#         x = torch.cat((cls_tokens, x), dim=1)  # (batch_size, 1 + num_patches, embed_dim)

#         x = x + self.pos_embed

#         for block in self.blocks:
#             x = block(x)

#         x = self.norm(x)
#         return x[:, 1:, :]  # discard the cls token, return only patches

In [11]:
# # Instantiate real vision encoder
# vision_encoder = TinyVisionTransformer(
#     img_size=224,
#     patch_size=16,
#     in_channels=3,
#     embed_dim=768,
#     depth=6,
#     num_heads=8
# )

# # Forward
# dummy_image = torch.randn(2, 3, 224, 224)  # (batch_size=2)
# vision_features = vision_encoder(dummy_image)  # (2, 196, 768)

# print(vision_features.shape)  # Should be (batch_size, 14x14=196 patches, 768 dim)

In [12]:
# torch.cuda.empty_cache()

In [13]:
# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl", use_fast = True)
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device).eval()

# # # --- Step 2: Define a simple RMBL ---
# # class SimpleRMBL(torch.nn.Module):
# #     def __init__(self, dim=256, num_memory_tokens=4, heads=4):
# #         super().__init__()
# #         self.memory = torch.nn.Parameter(torch.randn(1, num_memory_tokens, dim))
# #         self.transformer = torch.nn.TransformerEncoder(
# #             torch.nn.TransformerEncoderLayer(d_model=dim, nhead=heads),
# #             num_layers=1
# #         )

# #     def forward(self, segment_feat, memory=None):
# #         if memory is None:
# #             memory = self.memory.expand(segment_feat.size(0), -1, -1)
# #         combined = torch.cat([memory, segment_feat], dim=1)
# #         out = self.transformer(combined)
# #         return out[:, :memory.size(1)], out[:, memory.size(1):]


# # rmb_layer = SimpleRMBL(dim=256).to(device)
# # projector = torch.nn.Linear(2048, 256).to(device) 

# # # --- Step 4: Segment-Level Description with Memory ---
# # def describe_segments_with_memory(segments, initial_prompt):
# #     memory = None
# #     all_descriptions = []
# #     prompt = initial_prompt

# #     for idx, frames in enumerate(segments):
# #         n = len(frames)
# #         frame_indices = [int(n * 0.25), int(n * 0.5), int(n * 0.75)]
# #         selected_frames = [frames[i] for i in frame_indices if i < n]
# #         segment_description = []

# #         for image in selected_frames:
# #             inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)
# #             outputs = model.generate(**inputs, max_new_tokens=30)
# #             description = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
# #             segment_description.append(description)
# #             segment_description = filter_similar_descriptions(segment_description)

# #         all_descriptions.append(" ".join(segment_description))
# #         desc_inputs = processor.tokenizer(segment_description, return_tensors="pt", padding=True).to(device)
# #         with torch.no_grad():
# #             encoded = model.language_model.get_input_embeddings()(desc_inputs.input_ids) 
# #             pooled = encoded.mean(dim=1).unsqueeze(1)  
# #             pooled_projected = projector(pooled)    

# #         memory, _ = rmb_layer(pooled_projected, memory)

# #     return all_descriptions, memory

# def describe_segments_without_memory(segments, initial_prompt):
#     all_descriptions = []
#     prompt = initial_prompt

#     for idx, frames in enumerate(segments):
#         # Randomly select one frame from the segment
#         image = random.choice(frames)

#         inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)
#         outputs = model.generate(**inputs, max_new_tokens=30)
#         description = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)

#         all_descriptions.append(description.strip())

#     return all_descriptions


# # def summarize_video(memory, final_prompt, image):
# #     pooled_memory = memory.mean(dim=1)  # (B, 256)

# #     # Project memory back to match text embedding dim (approximation)
# #     memory_proj = projector.weight.T @ pooled_memory.T  # (2048, B)
# #     memory_proj = memory_proj.T  # (B, 2048)

# #     inputs = processor(images=image, text=final_prompt, return_tensors="pt").to(device)
# #     with torch.no_grad():
# #         outputs = model.generate(**inputs, max_new_tokens=50)

# #     return processor.tokenizer.decode(outputs[0], skip_special_tokens=True)

# prompt = "Describe what is happening in this segment in detail."
# final_prompt = "When did the person in the video put the straw into the bottle? Please state your answer with a brief explanation."

# # descriptions, final_memory = describe_segments_with_memory(segments, prompt)
# descriptions = describe_segments_without_memory(segments, prompt)
# # summary = summarize_video(final_memory, final_prompt, segments[-1][len(segments[-1]) // 2])

# print("Segment-level descriptions:")
# for i, d in enumerate(descriptions):
#     print(f"[Segment {i+1}]: {d}")

# # print("\nFinal Summary:", summary)

# context = " ".join(desc.strip().rstrip(".") + "." for desc in descriptions)
# full_input = f"This the description of the video: \n{context} \n{final_prompt}"

# inputs = processor(text=full_input, return_tensors="pt")

# outputs = model.language_model.generate(**inputs, max_new_tokens=100)
# print(processor.tokenizer.decode(outputs[0], skip_special_tokens=True))